In [1]:
import sys, os
ROOT = os.path.abspath("..")   # go up one directory from notebooks/
if ROOT not in sys.path:
    sys.path.append(ROOT)

print(ROOT)  


d:\PINN


In [2]:
from src2.networks import get_all_networks
from src2.config import CONFIG

from src2.sampling import (
    sample_domain_points,
    sample_top_surface,
    sample_interface,
    sample_far_field
)

from src2.losses import total_loss

from src2.pde_residuals import (
    residual_layer_coupled,
    residual_halfspace
)

from src2.boundary_conditions import (
    top_surface_bc,
    interface_layer_halfspace,
    halfspace_far_field_bc
)


In [3]:
import torch

In [4]:
model = torch.load('dispersion_pinn.pth')

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


'cpu'

In [10]:
# Load trained model weights
checkpoint = torch.load('dispersion_pinn.pth')

In [13]:
model_layer, model_half = get_all_networks()
model_layer.to(DEVICE)
model_half.to(DEVICE)

checkpoint = torch.load('dispersion_pinn.pth')
model_layer.load_state_dict(checkpoint["model_layer"])
model_half.load_state_dict(checkpoint["model_half"])

print("✓ Models loaded from checkpoint")

✓ Models loaded from checkpoint


In [7]:
geom = CONFIG["GEOMETRY"]

params_layer = CONFIG["LAYER"]
params_half  = CONFIG["HALFSPACE"]
dispersion = []   # <-- DEFINE DISPERSION HERE


In [8]:
H = geom["H"]   # layer thickness
L = geom["L"]   # half-space depth


In [22]:
# z-grid for plotting / post-processing
H=10;L=100
z_layer = torch.linspace(-H, 0.0, 10).reshape(-1, 1).to(DEVICE)
z_half  = torch.linspace(0.0, L, 10).reshape(-1, 1).to(DEVICE)
z_layer, z_half


(tensor([[-10.0000],
         [ -8.8889],
         [ -7.7778],
         [ -6.6667],
         [ -5.5556],
         [ -4.4444],
         [ -3.3333],
         [ -2.2222],
         [ -1.1111],
         [  0.0000]]),
 tensor([[  0.0000],
         [ 11.1111],
         [ 22.2222],
         [ 33.3333],
         [ 44.4444],
         [ 55.5556],
         [ 66.6667],
         [ 77.7778],
         [ 88.8889],
         [100.0000]]))

In [35]:
with torch.no_grad():
    scale = 1e-2
    # scale = 10
    # Layer (complex amplitude)   
    V_layer = model_layer(z_layer)
    V_R = scale * V_layer[:, 0:1]
    V_I = scale * V_layer[:, 1:2]
    # Half-space (real amplitude)
    V_half = scale * model_half(z_half)

In [36]:
V_half

tensor([[-5.7370e-06],
        [-8.7731e-06],
        [-9.9190e-06],
        [-1.0265e-05],
        [-1.0394e-05],
        [-1.0447e-05],
        [-1.0470e-05],
        [-1.0479e-05],
        [-1.0484e-05],
        [-1.0486e-05]])

In [37]:
V_layer

tensor([[-0.0001,  0.0003],
        [-0.0002,  0.0004],
        [-0.0002,  0.0004],
        [-0.0003,  0.0004],
        [-0.0004,  0.0004],
        [-0.0005,  0.0005],
        [-0.0006,  0.0005],
        [-0.0009,  0.0006],
        [-0.0011,  0.0006],
        [-0.0009,  0.0006]])